# A rough attempt to match receipt to record using bag of words

In [1]:
import pandas as pd
import numpy as np

from util_bill import *

## Read data

In [7]:
users = pd.read_csv("Bill.com/Users.csv")

# add a prediction column
users["prediction"] = [0 for i in range(len(users))]

import os
filesList = os.listdir("./Bill.com/ocr/")
file_id_list = [s[:-4] for s in filesList]

file_id_list.remove(".DS_S") # this one is not csv

def read_ocr(file):
    '''read one orc file into pandas dataframe'''

    data = {"x1":[], "y1":[],"x2":[], "y2":[],"x3":[], "y3":[],"x4":[], "y4":[], "value":[]}
    with open(file) as f:
         lines = f.readlines()

         for line in lines:
            row = line.split(',', 8)

            data['x1'].append(row[0])
            data['y1'].append(row[1])
            data['x2'].append(row[2])
            data['y2'].append(row[3])
            data['x3'].append(row[4])
            data['y3'].append(row[5])
            data['x4'].append(row[6])
            data['y4'].append(row[7])
            data['value'].append(row[8])

    return pd.DataFrame(data)

## Get bag of words for each row in users

In [3]:
def get_bag_of_words(strings):
  bag = []
  for sent in strings:
    bag += sent.lower().split(" ")
  return bag

BOW_users = []

for i in range(len(users)):
    temp_strings = [users.iloc[i][s] for s in ["vendor_name","vendor_address","date"]]
    BOW_users.append(get_bag_of_words(temp_strings))

## Define the bag of words similarity

In [4]:
def jaccard_similarity(x,y):
  """
  returns the jaccard similarity between two lists
  
  copied from https://newscatcherapi.com/blog/ultimate-guide-to-text-similarity-with-python
  """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)


In [9]:
def predict_one(file_id, path = "./Bill.com/ocr/", BOW_users = BOW_users, users_df = users):
    file = path+ file_id + ".csv"
    value_df = read_ocr(file)["value"]
    bag_strings = []
    for i in range(len(value_df)):
        bag_strings.append(value_df.iloc[i][:-1])
    BOW_file = get_bag_of_words(bag_strings)

    similarity = []
    for i in range(len(BOW_users)):
        similarity.append(jaccard_similarity(BOW_file, BOW_users[i]))

    max_loc = np.argmax(similarity)
    # print(max_loc, file_id)
    users_df.loc[max_loc,["prediction"]] = file_id

## Make the prediction

In [10]:
%%time

for file_id in file_id_list:
    predict_one(file_id)

CPU times: user 2.93 s, sys: 68.8 ms, total: 2.99 s
Wall time: 3.53 s


In [12]:
def calculate_accuracy(df):
    '''
    calculate the fraction of correction prediction

    df should have a column named "document"
    '''
    num_correct = np.sum(df["prediction"] == df["documentid"])
    return float(num_correct/len(df))

print("Accuracy: ", calculate_accuracy(users))

Accuracy:  0.29458917835671344
